In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import gkey
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [3]:
Base.prepare(engine, reflect=True)

In [4]:
session = Session(engine)

In [6]:
#create dataframe
brew_df = pd.read_sql_query("SELECT * FROM breweries", con=engine)
brew_df

,level_0,index,id,name,brewery_type,street,city,state,postal_code,country,longitude,latitude,phone,website_url,updated_at,zipcode,google_rating,google_numratings
0,0,0,5016,Bark Brewing Company,brewpub,3021 Spring Garden St,Greensboro,North Carolina,27403-1968,United States,-79.8436043406271,36.0620103554184,3368561406,http://barkbrewingcompany.com,2018-08-24T15:41:02.410Z,27403,3.5,4
1,1,1,5033,Blowing Rock Brewing Company,brewpub,152 Sunset Dr,Blowing Rock,North Carolina,28605-7205,United States,-81.677229,36.1327196,8284149600,http://www.blowingrockbrewing.com,2018-08-24T15:41:10.512Z,28605,4.2,241
2,2,2,5035,Blue Ghost Brewing Company,brewpub,125 Underwood Rd,Fletcher,North Carolina,28732-,United States,-82.530580304217,35.4368388,8283760159,http://www.blueghostbrewing.com,2018-08-24T15:41:11.489Z,28732,4.7,206
3,3,3,5067,Bull Durham Beer Co,micro,409 Blackwell St,Durham,North Carolina,27701-3972,United States,-78.90481567784,35.9915653,9197443568,http://www.bulldurhambeer.com,2018-08-24T15:41:22.982Z,27701,4.2,6
4,4,4,5090,Dirtbag Ales Brewery and Taproom,micro,3623 Legion Rd,Hope Mills,North Carolina,28348-8413,United States,-78.9220969691311,34.9939752576824,9104262537,http://www.dirtbagales.com,2018-08-24T15:41:33.473Z,28348,4.7,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,261,330,5273,Thirsty Monk Brewery,brewpub,92 Patton Ave,Asheville,North Carolina,28801-3315,United States,-82.5553147520814,35.59410425,8282545470,http://www.monkpub.com,2018-08-24T15:42:51.462Z,28801,4.6,293
262,262,331,5283,Triskelion Brewing Company,micro,340 7th Avenue East,Hendersonville,North Carolina,28792,United States,-82.4578969903109,35.3206001746298,8283887051,http://www.triskelionbrewing.com,2018-08-24T15:42:55.346Z,28792,4.7,166
263,263,332,5293,Valley River Brewery & Eatery,brewpub,71 Tennessee St,Murphy,North Carolina,28906-2938,United States,-84.0352081706441,35.0889040369639,8288372337,http://www.valleyriverbrewery.com,2018-08-24T15:42:59.685Z,28906,4.4,696
264,264,333,5304,White Street Brewing Company,micro,218 S White St,Wake Forest,North Carolina,27587-2742,United States,-78.5100617,35.9774896,9196479439,http://www.whitestreetbrewing.com,2018-08-24T15:43:05.047Z,27587,4.6,185


In [23]:
# load in NC brewery data CSV
nc_commerce_csv = "NC-Breweries_3-24-2020_clean-data.csv"

# make into a data frame
nc_breweries = pd.read_csv(nc_commerce_csv)
nc_breweries

,Company Name,street,City,State,Zip Code,Zip Plus,Contact First Name,Contact Last Name,Contact Title,Phone,...,NAICS2,NAICS3,NAICS4,NAICS5,Business Type,County,Employee,Annual Sales,URL,Year Established
0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,2529916958,...,NaN,NaN,NaN,NaN,Private,Wilson,14,10453000.0,217BREWWORKS.COM,2016
1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,NaN,NaN,NaN,8282259782,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,NaN,2018
2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,NaN,NaN,NaN,8284123212,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,NaN,2017
3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,NaN,NaN,NaN,8283212006,...,NaN,NaN,NaN,NaN,Private,Cherokee,14,17916000.0,NaN,2018
4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,3368350103,...,NaN,NaN,NaN,NaN,Private,Surry,12,16648000.0,ANGRYTROLLBREWING.COM,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Woodfin ABC,142 Weaverville Rd,Asheville,NC,28804,1238.0,Tammy,Ballew,Manager,8286589336,...,NaN,NaN,NaN,NaN,Private,Buncombe,5,2240000.0,WOODFINABC.NET,2007
909,Worth the Wine Co,1076 Weddington Woods St NW,Concord,NC,28027,8143.0,Lindsay G,Hitt,Owner,7040001111,...,NaN,NaN,NaN,NaN,Private,Cabarrus,2,465000.0,NaN,2016
910,Ws Wine Market LLC,1231 Creekshire Way,Winston Salem,NC,27103,3057.0,Elizabeth A,Binder,Manager,3368936950,...,NaN,NaN,NaN,NaN,Private,Forsyth,1,482000.0,NaN,2017
911,Youngsville ABC Store,101 N College St,Youngsville,NC,27596,NaN,Laurie,Leedy,Manager,9195563590,...,NaN,NaN,NaN,NaN,Private,Franklin,3,654000.0,NaN,2002


In [24]:
nc_breweries_tomerge = nc_breweries[['Company Name','street','Employee','Annual Sales','Year Established']]
nc_breweries_tomerge

,Company Name,street,Employee,Annual Sales,Year Established
0,217 Brew Works,217 South St S,14,10453000.0,2016
1,All Sevens Brewing,777 Haywood Rd,23,34511000.0,2018
2,Altamont Brewing Co,1050 Haywood Rd,23,34511000.0,2017
3,Andrews Brewing Co,1026 Main St,14,17916000.0,2018
4,Angry Troll Brewing,222 E Main St # U2,12,16648000.0,2015
...,...,...,...,...,...
908,Woodfin ABC,142 Weaverville Rd,5,2240000.0,2007
909,Worth the Wine Co,1076 Weddington Woods St NW,2,465000.0,2016
910,Ws Wine Market LLC,1231 Creekshire Way,1,482000.0,2017
911,Youngsville ABC Store,101 N College St,3,654000.0,2002


In [27]:
# merge datasets
brew_merge = pd.merge(brew_df, nc_breweries_tomerge, on="street", how="left")
brew_merge

,id,name,brewery_type,street,city,state,postal_code,country,longitude,latitude,phone,website_url,updated_at,zipcode,google_rating,google_numratings,Company Name,Employee,Annual Sales,Year Established
0,5016,Bark Brewing Company,brewpub,3021 Spring Garden St,Greensboro,North Carolina,27403-1968,United States,-79.8436043406271,36.0620103554184,3368561406,http://barkbrewingcompany.com,2018-08-24T15:41:02.410Z,27403,3.5,4,NaN,NaN,NaN,NaN
1,5033,Blowing Rock Brewing Company,brewpub,152 Sunset Dr,Blowing Rock,North Carolina,28605-7205,United States,-81.677229,36.1327196,8284149600,http://www.blowingrockbrewing.com,2018-08-24T15:41:10.512Z,28605,4.2,241,NaN,NaN,NaN,NaN
2,5035,Blue Ghost Brewing Company,brewpub,125 Underwood Rd,Fletcher,North Carolina,28732-,United States,-82.530580304217,35.4368388,8283760159,http://www.blueghostbrewing.com,2018-08-24T15:41:11.489Z,28732,4.7,206,Blue Ghost Brewing,14.0,20693000.0,2016.0
3,5067,Bull Durham Beer Co,micro,409 Blackwell St,Durham,North Carolina,27701-3972,United States,-78.90481567784,35.9915653,9197443568,http://www.bulldurhambeer.com,2018-08-24T15:41:22.982Z,27701,4.2,6,NaN,NaN,NaN,NaN
4,5090,Dirtbag Ales Brewery and Taproom,micro,3623 Legion Rd,Hope Mills,North Carolina,28348-8413,United States,-78.9220969691311,34.9939752576824,9104262537,http://www.dirtbagales.com,2018-08-24T15:41:33.473Z,28348,4.7,160,Dirtbag Ales,4.0,946000.0,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,5273,Thirsty Monk Brewery,brewpub,92 Patton Ave,Asheville,North Carolina,28801-3315,United States,-82.5553147520814,35.59410425,8282545470,http://www.monkpub.com,2018-08-24T15:42:51.462Z,28801,4.6,293,Thirsty Monk Pub Brewery,15.0,22507000.0,2008.0
263,5283,Triskelion Brewing Company,micro,340 7th Avenue East,Hendersonville,North Carolina,28792,United States,-82.4578969903109,35.3206001746298,8283887051,http://www.triskelionbrewing.com,2018-08-24T15:42:55.346Z,28792,4.7,166,NaN,NaN,NaN,NaN
264,5293,Valley River Brewery & Eatery,brewpub,71 Tennessee St,Murphy,North Carolina,28906-2938,United States,-84.0352081706441,35.0889040369639,8288372337,http://www.valleyriverbrewery.com,2018-08-24T15:42:59.685Z,28906,4.4,696,Valley River Brewery,6.0,7678000.0,2015.0
265,5304,White Street Brewing Company,micro,218 S White St,Wake Forest,North Carolina,27587-2742,United States,-78.5100617,35.9774896,9196479439,http://www.whitestreetbrewing.com,2018-08-24T15:43:05.047Z,27587,4.6,185,White Street Brewing Co,4.0,1761000.0,2012.0


In [36]:
brew_merge[brew_merge['Year Established'].isnull()]

,id,name,brewery_type,street,city,state,postal_code,country,longitude,latitude,phone,website_url,updated_at,zipcode,google_rating,google_numratings,Company Name,Employee,Annual Sales,Year Established
0,5016,Bark Brewing Company,brewpub,3021 Spring Garden St,Greensboro,North Carolina,27403-1968,United States,-79.8436043406271,36.0620103554184,3368561406,http://barkbrewingcompany.com,2018-08-24T15:41:02.410Z,27403,3.5,4,NaN,NaN,NaN,NaN
1,5033,Blowing Rock Brewing Company,brewpub,152 Sunset Dr,Blowing Rock,North Carolina,28605-7205,United States,-81.677229,36.1327196,8284149600,http://www.blowingrockbrewing.com,2018-08-24T15:41:10.512Z,28605,4.2,241,NaN,NaN,NaN,NaN
3,5067,Bull Durham Beer Co,micro,409 Blackwell St,Durham,North Carolina,27701-3972,United States,-78.90481567784,35.9915653,9197443568,http://www.bulldurhambeer.com,2018-08-24T15:41:22.982Z,27701,4.2,6,NaN,NaN,NaN,NaN
5,5099,Eleven Lakes Brewing Company,micro,10228 Bailey Rd Ste 201,Cornelius,North Carolina,28031-9420,United States,-80.8605492789893,35.4540068565198,7044000927,http://www.elevenlakesbrewing.com,2018-08-24T15:41:38.237Z,28031,4.8,112,NaN,NaN,NaN,NaN
7,5121,Ghost Harbor Brewing Company,micro,606 B E Colonial Ave,Elizabeth City,North Carolina,27909,United States,-76.2187962244898,36.3004222857143,2525991030,http://www.ghostharborbrewing.com,2018-08-24T15:41:47.030Z,27909,4.8,133,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,5316,York Chester Brewing Company,micro,1500 River Dr Ste 250,Belmont,North Carolina,28012-3587,United States,-81.0156159,35.236705,7047555104,http://www.yorkchesterbrew.com,2018-08-11T21:38:54.451Z,28012,4.9,56,NaN,NaN,NaN,NaN
260,5205,Newgrass Brewing Company,micro,213 S Lafayette St,Shelby,North Carolina,28150-5349,United States,-81.5403575555556,35.2906197777778,7049371280,http://www.newgrassbrewing.com,2018-08-24T15:42:21.589Z,28150,4.6,353,NaN,NaN,NaN,NaN
261,5232,Quick Trigger Brewing Company,brewpub,118 Main St W,Ahoskie,North Carolina,27910-3302,United States,-76.9863065,36.2857593,2523326844,http://www.quicktriggerbrewing.com,2018-08-24T15:42:31.190Z,27910,4.8,145,NaN,NaN,NaN,NaN
263,5283,Triskelion Brewing Company,micro,340 7th Avenue East,Hendersonville,North Carolina,28792,United States,-82.4578969903109,35.3206001746298,8283887051,http://www.triskelionbrewing.com,2018-08-24T15:42:55.346Z,28792,4.7,166,NaN,NaN,NaN,NaN


In [28]:
# add data to sqlite database
brew_merge.to_sql('breweries', con=engine, if_exists='replace')

In [29]:
#test sqlite data
engine.execute("SELECT * FROM breweries").fetchall()[0]

(0, 5016, 'Bark Brewing Company', 'brewpub', '3021 Spring Garden St', 'Greensboro', 'North Carolina', '27403-1968', 'United States', '-79.8436043406271', '36.0620103554184', '3368561406', 'http://barkbrewingcompany.com', '2018-08-24T15:41:02.410Z', '27403', '3.5', 4, None, None, None, None)

In [34]:
# create sqlite database of naics data
nc_breweries_database = nc_breweries[nc_breweries['Primary NAICS'] == 312120]
# add data to sqlite database
nc_breweries_database.to_sql('business', con=engine, if_exists='replace')

In [35]:
#test sqlite data
engine.execute("SELECT * FROM business").fetchall()[0]

(0, '217 Brew Works', '217 South St S', 'Wilson', 'NC', 27893, 4911.0, 'Tom', 'Curran', 'Owner', 2529916958, 'Brewers (mfrs)', 312120, None, None, None, None, 'Private', 'Wilson', 14, 10453000.0, '217BREWWORKS.COM', 2016)